# 0. Introduction
1. We write `DeepmdSeTildeR` to construct the $\widetilde{R}$ in `DeepPot-SE`

In [1]:
from timeit import default_timer as timer
from matersdk.io.publicLayer.structure import DStructure
from matersdk.io.publicLayer.neigh import (
                StructureNeighborsDescriptor,
                StructureNeighborsUtils
)
from matersdk.feature.deepmd.se_pair import DpseTildeRPairDescriptor

import warnings
warnings.filterwarnings('ignore')


# 1. Demo for `DpseTildeRPairV1`

In [3]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]
reformat_mark = True
coords_are_cartesian = True

center_atomic_number = 42
nbr_atomic_number = 42
rcut = 3.3
rcut_smooth = 3.0

structure = DStructure.from_file(
                file_format="pwmat",
                file_path=atom_config_path)
neighbors = StructureNeighborsDescriptor.create(
                'v1',
                structure=structure,
                rcut=rcut,
                scaling_matrix=scaling_matrix,
                reformat_mark=reformat_mark,
                coords_are_cartesian=coords_are_cartesian)

### Step 1. Print the attributions of DeepmdSeR
dpse_tildeR_pair = DpseTildeRPairDescriptor.create(
                'v1',
                structure_neighbors=neighbors,
                center_atomic_number=center_atomic_number,
                nbr_atomic_number=nbr_atomic_number,
                rcut=rcut,
                rcut_smooth=rcut_smooth)

print()
print("Step 1. Print the attributions of DeepmdSeR:")
print("\t1. deepmd_se_r.dp_feature_pair_an.shape = ", dpse_tildeR_pair.dp_feature_pair_an.shape)
print("\t2. deepmd_se_r.dp_feature_pair_d.shape = ", dpse_tildeR_pair.dp_feature_pair_d.shape)
print("\t3. deepmd_se_r.dp_feature_pair_rc.shape = ", dpse_tildeR_pair.dp_feature_pair_rc.shape)


### Step 2. Get smooth edition s_{ij}
print()
print("Step 2. Get segmented form of s in Deepmd:")
print("\t1. s.shape = ", dpse_tildeR_pair._get_s(rcut=rcut, rcut_smooth=rcut_smooth).shape)


### Step 3.
print()
print("Step 3.")
print("\t1. deepmd_se_r.dp_feature_pair_tildeR.shape = ", dpse_tildeR_pair.dp_feature_pair_tildeR.shape)


Step 1. Print the attributions of DeepmdSeR:
	1. deepmd_se_r.dp_feature_pair_an.shape =  (4, 6)
	2. deepmd_se_r.dp_feature_pair_d.shape =  (4, 6)
	3. deepmd_se_r.dp_feature_pair_rc.shape =  (4, 6, 3)

Step 2. Get segmented form of s in Deepmd:
	1. s.shape =  (4, 6)

Step 3.
	1. deepmd_se_r.dp_feature_pair_tildeR.shape =  (4, 6, 4)


In [4]:
print("Step 4. With zero-padding, the shape of tildeR = ", end=' ')
print(dpse_tildeR_pair.get_tildeR(max_num_nbrs=14).shape)

Step 4. With zero-padding, the shape of tildeR =  (4, 14, 4)


In [5]:
dpse_tildeR_pair.dp_feature_pair_tildeR

array([[[ 0.08146538,  0.08146538,  0.        ,  0.        ],
        [ 0.08146538, -0.08146538,  0.        ,  0.        ],
        [ 0.08146538, -0.04073269,  0.07055109,  0.        ],
        [ 0.08146538,  0.04073269, -0.07055109,  0.        ],
        [ 0.08146509,  0.04073255,  0.07055083,  0.        ],
        [ 0.08146509, -0.04073255, -0.07055083,  0.        ]],

       [[ 0.08146538, -0.08146538,  0.        ,  0.        ],
        [ 0.08146538,  0.08146538,  0.        ,  0.        ],
        [ 0.08146538, -0.04073269,  0.07055109,  0.        ],
        [ 0.08146538,  0.04073269, -0.07055109,  0.        ],
        [ 0.08146509,  0.04073255,  0.07055083,  0.        ],
        [ 0.08146509, -0.04073255, -0.07055083,  0.        ]],

       [[ 0.08146538, -0.08146538,  0.        ,  0.        ],
        [ 0.08146538,  0.08146538,  0.        ,  0.        ],
        [ 0.08146538,  0.04073269, -0.07055109,  0.        ],
        [ 0.08146538, -0.04073269,  0.07055109,  0.        ],
    

# 2. For PWmatMLFF, get `neigh_list`

In [6]:
neigh_list = StructureNeighborsUtils.get_nbrs_indices(
                    struct_nbr=neighbors,
                    center_atomic_number=center_atomic_number,
                    nbr_atomic_number=nbr_atomic_number)
print(neigh_list)

TypeError: get_nbrs_indices() got an unexpected keyword argument 'center_atomic_number'

# 3. Calculate the gradient of $\tilde{R}^i_j$ with respect to `x`, `y` and `z`

In [9]:
### Step 1.
tildeR_deriv = dpse_tildeR_pair._calc_derivative()
print("1. Without zero-padding, shape of gradient = ", tildeR_deriv.shape)

### Step 2. 
#tildeR_deriv = dpse_tildeR_pair.calc_derivative(max_num_nbrs=26)
#print("1. With zero-padding, shape of gradient = ", tildeR_deriv.shape)

1. Without zero-padding, shape of gradient =  (4, 6, 4, 3)


In [10]:
tildeR_deriv

array([[[[ 1.71177981,  0.        ,  0.        ],
         [ 1.71177981,  0.        ,  0.        ],
         [ 0.        , -0.02553521,  0.        ],
         [ 0.        ,  0.        , -0.02553521]],

        [[-1.71177981,  0.        ,  0.        ],
         [ 1.71177981, -0.        , -0.        ],
         [-0.        , -0.02553521,  0.        ],
         [-0.        ,  0.        , -0.02553521]],

        [[-0.85588981,  1.48244486,  0.        ],
         [ 0.40879345, -0.75227942, -0.        ],
         [-0.75227942,  1.27745115,  0.        ],
         [-0.        ,  0.        , -0.02553521]],

        [[ 0.85588981, -1.48244486,  0.        ],
         [ 0.40879345, -0.75227942,  0.        ],
         [-0.75227942,  1.27745115, -0.        ],
         [ 0.        , -0.        , -0.02553521]],

        [[ 0.85588874,  1.48244267,  0.        ],
         [ 0.40879305,  0.75227841,  0.        ],
         [ 0.75227841,  1.27744921,  0.        ],
         [ 0.        ,  0.        , -0.025